In [8]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

In [9]:
def make_pipe_clf(scaler,feature_selection,decomp,clf,order):
# order = 1 : first perform feature selection and then apply PCA
# order = 0 : first apply PCA and then reduce the transformed features
    if order:
        pipeline = Pipeline([('scaler', scaler),
                    ('feature_selection', feature_selection),
                    ('decomp', decomp),         
                    ('classifier', clf) ])
    else:
        pipeline = Pipeline([('scaler', scaler),
                    ('decomp', decomp ),                 
                    ('feature_selection', feature_selection),        
                    ('classifier', clf) ])
    return pipeline
###########################################################################################
def make_pipe(scaler,feature_selection,decomp,order):
    if order:
        pipeline = Pipeline([('scaler', scaler),
                    ('feature_selection', feature_selection),
                    ('decomp', decomp),         
                     ])
    else:
        pipeline = Pipeline([('scaler', scaler),
                    ('decomp', decomp ),                 
                    ('feature_selection', feature_selection),        
                     ])
    return pipeline

In [10]:
class Nested_cv:
    def __init__(self, n_outer_folds = 3, n_inner_folds = 3, n_top =1, state = 42):
        self.n_outer_folds = n_outer_folds
        self.n_inner_folds = n_inner_folds
        self.n_top = n_top
        self.state = 42
                                        
        
    def set_pipe(self, pip_steps, pip_params):
        n_steps = len(pip_steps)
        if n_steps == 4:
            self.pipe = make_pipe(scaler = pip_steps[0], feature_selection = pip_steps[1], decomp = pip_steps[2], order = pip_steps[3])
            self.params = pip_params
        if n_steps == 5:
            self.pipe = make_pipe_clf(scaler = pip_steps[0], feature_selection = pip_steps[1], decomp = pip_steps[2], order = pip_steps[4], clf = pip_steps[3])
            self.params = pip_params
        else:
            print ("Number of steps gotta be either 4 or 5, you inserted %d." %n_steps)
        
    def fit(self, x_tot, y_tot, verbose = 0):
        self.outer_cv = StratifiedKFold(n_splits = self.n_outer_folds, shuffle = True, random_state = self.state)
        self.inner_cv = StratifiedKFold(n_splits = self.n_inner_folds, shuffle = True, random_state = self.state)
        self.grid = GridSearchCV(self.pipe, self.params, cv = self.inner_cv, verbose = verbose)       
        self.score = cross_val_score(self.grid, X = x_tot, y = y_tot, cv = self.outer_cv, verbose = verbose)
    
    def get_score(self):
        # elegxo gia to unfitted
        return self.score
    def get_mean_score(self):
        self.mean_score = self.score.mean()
        print ("Mean score of %d outer folds : %f" %(self.n_outer_folds, self.mean_score ))
        return self.mean_score
    
        

In [13]:
# ====== Example
from sklearn.datasets import load_iris
iris = load_iris()
X_iris = iris.data
y_iris = iris.target

nest2 = Nested_cv(n_outer_folds= 5, n_inner_folds =5, n_top=1)
params = {'feature_selection__k' : [3], 'decomp__n_components':[1], 'classifier__kernel' : ['rbf'], 'classifier__C': [10], 'classifier__gamma': [1]}
steps = [StandardScaler(), SelectKBest(), PCA(), SVC(), 1]
nest2.set_pipe(pip_steps = steps,pip_params=params)
nest2.fit(X_iris, y_iris, verbose = 0)
nest2.get_mean_score()
nest2.get_score()

Mean score of 5 outer folds : 0.893333


array([ 0.86666667,  0.93333333,  0.86666667,  0.9       ,  0.9       ])